In [1]:
from src.solvers import pg_solver
import os

from ray import tune, air
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.optuna import OptunaSearch
import optuna

############################################
# Experiment
############################################

instance_dir = os.path.abspath('data/sat_rand/sat_rand_n=0020_k=03_m=0080_i=1.cnf')
exp_name = 'exp_condit'
local_dir = "experiments"

# Assumptions
num_samples = 15000
transformer_assump = False
node2vec_assump = False
baseline_assump = False 

def define_by_run_func(trial):
    # Constants
    config = {
        # Regularization
        "early_stopping": False,
        "patience": 6,
        "entropy_value": 0,

        "log_interval": 50,
        "eval_interval": 50,
        "eval_strategies": [0, 32],
        "tensorboard_on": True,
        "extra_logging": False,  # log TrainableState's weights
        "raytune": True,
        "data_dir": instance_dir,
        "verbose": 0,  # {0, 1, 2}. If raytune is True, then verbose is set to 0.

        "log_dir": 'logs',
        "output_dir": 'outputs',
        "exp_name": exp_name,
        "run_name": 'run',
        "gpu": True,
        "checkpoint_dir": None} 

    # Search space
    # Input and embeddings
    dec_context_initializer = "EmptyContext"
    if node2vec_assump:
        config["dec_var_initializer"] = "Node2VecVar"
        dec_context_initializer = trial.suggest_categorical("dec_context_initializer", ["EmptyContext", "Node2VecContext"])
        config["node2vec"] = True  # {False, True}
        config["n2v_dir"] = os.path.abspath("n2v_emb")
        config["n2v_dim"] = 64
        config["n2v_pretrained"] = True  # {False, True}
        config["n2v_walk_len"] = 10
        config["n2v_context_size"] = 5
        config["n2v_walks_per_node"] = 5
        config["n2v_p"] = 1
        config["n2v_q"] = 1
        config["n2v_batch_size"] = 32
        config["n2v_lr"] = 0.01
        config["n2v_num_epochs"] = 100
        config["n2v_workers"] = 0  # {0, 1, 2, 3, 4}
        config["n2v_verbose"] = 0  # {0, 1, 2}
    else:
        config["node2vec"] = False
        config["dec_var_initializer"] = "BasicVar"
        config["dec_context_initializer"] = "EmptyContext"
    trial.suggest_categorical("var_emb_size", [64, 128, 256])
    trial.suggest_categorical("assignment_emb_size", [64, 128, 256])
    if dec_context_initializer == "Node2VecContext":
        trial.suggest_categorical("context_emb_size", [64, 128, 256])
    else: # EmptyContext
        config["context_emb_size"] = 0
    trial.suggest_categorical("model_dim", [64, 128, 256, 512])

    # Decoder
    if transformer_assump:
        config["decoder"] = "Transformer"
        trial.suggest_int("num_heads", 1, 8)
        trial.suggest_categorical("dense_size", [64, 128, 256, 512, 768, 1024])
    else:  # rnn decoder
        trial.suggest_categorical("decoder", ['GRU', 'LSTM'])
        trial.suggest_categorical("hidden_size", [64, 128, 256, 512, 768, 1024])
        trial.suggest_categorical("trainable_state", [True, False])
    trial.suggest_int("num_layers", 1, 6, log=True)
    trial.suggest_categorical("output_size", [1, 2])
    trial.suggest_float("dropout", 0, 0.3, step=0.05)

    # Training
    config["num_samples"] = num_samples
    trial.suggest_int("accumulation_episodes", 1, 5)
    trial.suggest_categorical("batch_size", [2, 4, 8, 16, 32, 64])
    config["permute_vars"] = True
    config["permute_seed"] = None  # 2147483647
    trial.suggest_categorical("clip_grad", [None, 0.5, 1, 1.5, 2])
    trial.suggest_float("lr", 1e-6, 1e-4, log=True)  # 0.00015   0.00001
    
    # Baseline
    if baseline_assump:
        baseline = trial.suggest_categorical("baseline", ["greedy", "sample", "ema"])
        if baseline == "sample":
            trial.suggest_categorical("k_samples", [2, 4, 8, 16, 32])  # int, k >= 1
        elif baseline == "ema":
            trial.suggest_float("alpha_ema", 0.95, 0.99, step=0.01)  # 0 <= alpha <= 1
    else:  # No baseline
        config["baseline"] = None

    # Exploration
    trial.suggest_categorical("logit_clipping", [None, 1, 2, 5, 10])  # {None, int >= 1}
    trial.suggest_categorical("logit_temp", [None, 1.5, 2, 2.5])  # {None, int >= 1}
    trial.suggest_categorical("entropy_estimator", ['crude', 'smooth'])
    trial.suggest_categorical("beta_entropy", [0, 0.01, 0.02, 0.03])  # float, beta >= 0

    # Optuna defined by run flag. Don't set to False
    config["optuna_by_run"] = True
    return config
    

search_alg = OptunaSearch(sampler=optuna.samplers.TPESampler(n_startup_trials=10,
                                                             n_ei_candidates=24,
                                                             multivariate=False),
                          space=define_by_run_func,
                          mode='max',
                          metric="num_sat_sample_32")
search_alg = ConcurrencyLimiter(search_alg, max_concurrent=4)
scheduler = ASHAScheduler(grace_period=5)
tune_config = tune.TuneConfig(mode='max',
                              metric="num_sat_sample_32",
                              num_samples=5,
                              search_alg=search_alg,
                              scheduler=scheduler)
run_config = air.RunConfig(local_dir=local_dir,
                           name=exp_name,
                           progress_reporter=None,
                           log_to_file=True)

tuner = tune.Tuner(pg_solver,
                   tune_config=tune_config,
                   run_config=run_config)

results = tuner.fit()

(pg_solver pid=63704) /Users/omargutierrez/opt/miniconda3/envs/sat/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(pg_solver pid=63704)   warnings.warn('resource_tracker: There appear to be %d '


In [9]:
15000 / 7

2142.8571428571427

In [18]:
import numpy as np 
int(np.ceil(15000/7))

2143

In [ ]:
from src.solvers import pg_solver
import os

from ray import tune, air
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.optuna import OptunaSearch
import optuna


config = {
    # Encoder
    "node2vec": True,  # {False, True}
    "n2v_dir": "n2v_emb",
    "n2v_dim": 64,
    "n2v_pretrained": True,  # {False, True}
    "n2v_walk_len": 10,
    "n2v_context_size": 5,
    "n2v_walks_per_node": 5,
    "n2v_p": 1,
    "n2v_q": 1,
    "n2v_batch_size": 32,
    "n2v_lr": 0.01,
    "n2v_num_epochs": 100,
    "n2v_workers": 0,  # {0, 1, 2, 3, 4}
    "n2v_verbose": 1,  # {0, 1, 2}

    # Initializers
    "dec_var_initializer": "Node2VecVar",  # {"BasicVar", "Node2VecVar"}
    "dec_context_initializer": "Node2VecContext",  # {"EmptyContext", "Node2VecContext"}

    # Embeddings
    "var_emb_size": 128,
    "assignment_emb_size": 64,
    "context_emb_size": 128,
    "model_dim": 128, 

    # Architecture
    "decoder": 'GRU',  # {'GRU', 'LSTM', "Transformer"}
    "num_layers": 2,  
    "output_size": 2,  #Decoder output size: 1, 2
    "dropout": 0,

    "hidden_size": 128,  #hidden_size if RNN
    "trainable_state": True,  # {False, True}

    #"num_heads": 2,
    #"dense_size":128,

    # Training
    "num_episodes": 1500,  #4000
    "accumulation_episodes": 1,
    "batch_size": 10,  #10
    "permute_vars": True,
    "permute_seed": None,  # 2147483647
    "clip_grad": 1,
    "lr": 0.00001,  # 0.00015   0.00001

    # Baseline
    "baseline": 'sample',  # {None, 'greedy', 'sample'. 'ema'}
    "alpha_ema": 0.99,  # 0 <= alpha <= 1. EMA decay, useful if baseline == 'ema'
    "k_samples": 10,  # int, k >= 1. Number of samples used to obtain the baseline value, useful if baseline == 'sample'

    # Exploration
    "logit_clipping": None,  # {None, int >= 1}
    "logit_temp": None,  # {None, int >= 1}. Useful for improve exploration in evaluation.
    "entropy_estimator": 'crude',  # {'crude', 'smooth'}
    "beta_entropy": 0.03,  # float, beta >= 0.

    # Regularization
    "early_stopping": False,
    "patience": 6,
    "entropy_value": 0,

    "log_interval": 20,
    "eval_interval": 100,
    "eval_strategies": [0, 32],
    "tensorboard_on": True,
    "extra_logging": False,  # log TrainableState's weights
    "raytune": True,
    "data_dir": os.path.abspath('data/sat_rand/sat_rand_n=0020_k=03_m=0040_i=1.cnf'),
    "verbose": 0,

    "log_dir": 'logs',
    "output_dir": 'outputs',
    "exp_name": 'exp_tune',
    "run_name": 'run',
    "gpu": True,
    "checkpoint_dir": None} 

#config["decoder"] = tune.choice(['GRU', 'LSTM', 'Transformer'])
config["k_samples"] = tune.qrandint(2, 32, 2)  # Round to multiples of 2 (includes 32)
config["entropy_estimator"] = tune.choice(['crude', 'smooth'])
config["beta_entropy"] = tune.choice([0.01, 0.02, 0.03])
config["batch_size"] = tune.choice([1, 2, 4, 8, 16, 32, 64])
config["lr"] = tune.qloguniform(1e-6, 1e-4, 5e-7)  # Round to multiples of 0.0000005

def define_by_run_func(trial):
    decoder = trial.suggest_categorical('decoder', ['GRU', 'LSTM', 'Transformer'])

#search_alg = None
search_alg = OptunaSearch(sampler=optuna.samplers.TPESampler(multivariate=False))
search_alg = ConcurrencyLimiter(search_alg, max_concurrent=4)
scheduler = ASHAScheduler(grace_period=5)
tune_config = tune.TuneConfig(mode='max',
                              metric="num_sat_sample_32",
                              num_samples=10,
                              search_alg=search_alg,
                              scheduler=scheduler)
run_config = air.RunConfig(name="my-experiment-1",
                           local_dir="my_results",
                           progress_reporter=None)

tuner = tune.Tuner(pg_solver,
                   tune_config=tune_config,
                   run_config=run_config,
                   param_space=config)

results = tuner.fit()

In [ ]:
local_dir = "my_results"
exp_name = "my-experiment-1"
experiment_path = f"{local_dir}/{exp_name}"
print(f"Loading results from {experiment_path}...")

restored_tuner = tune.Tuner.restore(path=experiment_path,
                                    trainable=pg_solver)
result_grid = restored_tuner.get_results()

# Check if there have been errors
if result_grid.errors:
    print("One of the trials failed!")
else:
    print("No errors!")

num_results = len(result_grid)
print("Number of results:", num_results)

In [ ]:
# Get the result with the maximum test set `mean_accuracy`
best_result = result_grid.get_best_result(metric="num_sat_sample_32", mode="max")

# Best result’s hyperparameter configuration
print("\nBest result config:\n", best_result.config)

# Trial’s log directory 
print("\nBest result logdir:\n",best_result.log_dir)

# Get the last reported set of metrics
print("\nBest result metrics:\n", best_result.metrics)

In [ ]:
result_df = best_result.metrics_dataframe
result_df[["training_iteration", "episode", "num_sat_greedy", "num_sat_sample_32", "time_total_s"]]

In [ ]:
print("Best hyperparameters found were: ", results.get_best_result().config)

In [ ]:
# Obtain a trial dataframe from all run trials of this `tune.run` call.
dfs = {result.log_dir: result.metrics_dataframe for result in results}
dfs

In [ ]:
#[d.num_sat_sample_32.plot() for d in dfs.values()]
# Plot by epoch
ax = None  # This plots everything on the same plot
for d in dfs.values():
    ax = d.num_sat_sample_32.plot(ax=ax, legend=False)
ax.set_xlabel('Episodes')
ax.set_ylabel("Number of sat clauses");

In [ ]:
# Plot by epoch
ax = None  # This plots everything on the same plot
for d in dfs.values():
    ax = d.num_sat_greedy.plot(ax=ax, legend=False)
ax.set_xlabel('Episodes')
ax.set_ylabel("Number of sat clauses");

In [ ]:
logdir = results.get_best_result("num_sat_sample_32", mode="max").log_dir
logdir

In [ ]:
import os
import torch

logdir = results.get_best_result("num_sat_sample_32", mode="max").log_dir
#state_dict = torch.load(os.path.join(logdir, "model.pth"))

#model = ConvNet()
#model.load_state_dict(state_dict)